In [1]:
import numpy as np
import pandas as pd

In [9]:
data = pd.read_csv("../data/train.csv",encoding='windows-1251', sep=';', low_memory=False)

In [ ]:
from sklearn.model_selection import train_test_split

drop_cols = [
    'PastDueLess_60',
    'PastDue60_90',
    'PastDue90_More',
    'OpenLoans',
    'RealEstateLoans',
    'Income_inverse',
    'Age_inverse'
]

X = data.drop(['Target'] + drop_cols, axis=1)
y = data['Target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

NameError: name 'train_data' is not defined

In [ ]:
import optuna
import lightgbm as lgb
from sklearn.metrics import roc_auc_score, f1_score
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
# Определим функцию для оптимизации гиперпараметров с использованием Optuna
def objective(trial):
    param = {
        'objective': 'binary',
        'metric': 'auc',
        'boosting_type': trial.suggest_categorical('boosting_type', ['gbdt', 'dart']),
        'num_leaves': trial.suggest_int('num_leaves', 31, 256),
        'max_depth': trial.suggest_int('max_depth', -1, 15),
        'learning_rate': trial.suggest_float('learning_rate', 1e-5, 0.1, log=True),
        'n_estimators': trial.suggest_int('n_estimators', 50, 1000),
        'subsample': trial.suggest_uniform('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.5, 1.0),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'lambda_l1': trial.suggest_float('lambda_l1', 1e-5, 10, log=True),
        'lambda_l2': trial.suggest_float('lambda_l2', 1e-5, 10, log=True),
        'class_weight': trial.suggest_categorical('class_weight', [None, 'balanced'])
    }



    model = lgb.LGBMClassifier(**param)

    # Обучаем модель
    model.fit(X_train, y_train)

    # Предсказания на тестовых данных
    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test)[:, 1]

    # Метрики
    f1 = f1_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_prob)

    # Оптимизируем модель по ROC AUC
    return roc_auc

# Запуск оптимизации
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=250)

print("Best hyperparameters:", study.best_params)

# Лучшие параметры
best_params = study.best_params
# Строим модель с лучшими гиперпараметрами
best_model = lgb.LGBMClassifier(**best_params)
# Обучаем финальную модель на всех данных
best_model.fit(X, y)

# Оценка модели на тестовых данных
y_pred_final = best_model.predict(X_test)
y_prob_final = best_model.predict_proba(X_test)[:, 1]
final_f1 = f1_score(y_test, y_pred_final)
final_roc_auc = roc_auc_score(y_test, y_prob_final)

print("Final F1 Score:", final_f1)
print("Final ROC AUC:", final_roc_auc)


In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

fpr_train, tpr_train, _ = roc_curve(y_train, best_model.predict_proba(X_train)[:, 1])
roc_auc_train = auc(fpr_train, tpr_train)

fpr_test, tpr_test, _ = roc_curve(y_test, best_model.predict_proba(X_test)[:, 1])
roc_auc_test = auc(fpr_test, tpr_test)

plt.figure(figsize=(8, 6))
plt.plot(fpr_train, tpr_train, color='blue', label=f'Train ROC curve (area = {roc_auc_train:.4f})')
plt.plot(fpr_test, tpr_test, color='green', label=f'Test ROC curve (area = {roc_auc_test:.4f})')
plt.plot([0, 1], [0, 1], color='red', linestyle='--', label='Random guess')

plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve - Train vs Test')
plt.legend(loc='lower right')
plt.grid()
plt.show()

In [ ]:
importances = best_model.feature_importances_
features = X_train.columns
feature_importance_df = pd.DataFrame({'Feature': features, 'Importance': importances})
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

plt.figure(figsize=(10, 6))
sns.barplot(data=feature_importance_df.head(20), x='Importance', y='Feature', palette='viridis')
plt.title('ТОП важных признаков в модели LightGBM')
plt.tight_layout()
plt.show()